# Model Training

This notebook is used to train a model for the sketch detection task.

In [ ]:
!echo $PATH
import os

os.environ['PATH'] = os.environ['HOME'] + '/.local/bin:' + os.environ['PATH']
!echo $PATH

In [ ]:
!python3 --version

In [ ]:
!nvcc --version

In [ ]:
# @formatter:off
# https://pytorch.org/get-started/previous-versions/
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121 --user
!pip install 'git+https://github.com/facebookresearch/detectron2.git' --user
# @formatter:on

In [ ]:
import torch, torchvision

print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)
!gcc --version

In [ ]:
import os

from detectron2.utils.logger import setup_logger

setup_logger()

## Collect Training Data

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from src.utils.utils_json import print_json, read_json
from src.dataset.dataset import read_dateset_from

# @formatter:off
# datadir = "datasets/fa"
datadir = "datasets/hdBPMN-icdar2021"
# datadir = "Sketches-Dataset-main/data"
# @formatter:on

training_meta_path = os.path.join(datadir, "train.json")
classes = list(map(lambda x: x["name"], read_json(training_meta_path)["categories"]))
classes.sort()

print_json(classes, tag="Classes")


def on_register_dataset(x):
    dataset = read_dateset_from(datadir, x)
    return dataset


for d in ["train", "val"]:
    name = "sketches_" + d

    # Remove previously registered datasets if they exist
    try:
        DatasetCatalog.remove(name)
        MetadataCatalog.remove(name)
    except KeyError:
        pass

    # Register new dataset
    DatasetCatalog.register(name, lambda x=d: on_register_dataset(x))
    MetadataCatalog.get(name).set(thing_classes=classes)

In [ ]:
from detectron2.utils.visualizer import Visualizer
import cv2
import random

sketches_metadata = MetadataCatalog.get("sketches_train")
training_dataset = read_dateset_from(datadir, "train")

In [ ]:
# https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html#metadata-for-datasets
MetadataCatalog.get("sketches_train").keypoint_names = [
    "head",
    "tail",
]

MetadataCatalog.get("sketches_train").keypoint_flip_map = (
    ("head", "tail"),
    ("tail", "head"),
)

print_json(MetadataCatalog.get("sketches_train").thing_classes, tag="thing_classes")
print_json(MetadataCatalog.get("sketches_train").keypoint_names, tag="keypoint_names")

## Configure Model Training

In [ ]:
from names_generator import generate_name

model_name = generate_name()
print(f"Model name: '{model_name}'")

In [ ]:
import multiprocessing
from detectron2 import model_zoo
from detectron2.config import get_cfg

from src.utils.utils_json import write_json
from src.sketch_detection_rcnn.roi_heads import SketchROIHeads  # noqa # pylint: disable=unused-import

pretrained_model = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"

cfg = get_cfg()

# Use pre-trained model
cfg.merge_from_file(model_zoo.get_config_file(pretrained_model))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(pretrained_model)

# Use pre-pre-trained model
# cfg.merge_from_file(model_zoo.get_config_file(pretrained_model))
# cfg.MODEL.WEIGHTS = "/home/jupyter-patrickzierahn/models/clever_mahavira/model_final.pth"

cfg.OUTPUT_DIR = os.path.join("models", model_name)

# Set training data
cfg.DATALOADER.NUM_WORKERS = multiprocessing.cpu_count()
cfg.DATASETS.TRAIN = ("sketches_train",)
cfg.DATASETS.TEST = ("sketches_val",)

# pick a good LR
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.IMS_PER_BATCH = 10

# Region of Interest
cfg.MODEL.ROI_HEADS.NAME = "SketchROIHeads"
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)

# Keypoints
cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 2
cfg.TEST.KEYPOINT_OKS_SIGMAS = [1.0] * cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS

# Write config to file
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg_file = os.path.join(cfg.OUTPUT_DIR, "cfg.json")
write_json(cfg_file, cfg)

# Write class names to file
classes_file = os.path.join(cfg.OUTPUT_DIR, "classes.json")
write_json(classes_file, classes)

# Print config
print_json(cfg)

## Print Model Layers

In [ ]:
from detectron2.modeling import build_model

model = build_model(cfg)

## Train Model

In [ ]:
import time
from detectron2.engine import DefaultTrainer

start_time = time.time()

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

training_duration = time.time() - start_time

## Gather Evaluation Metrics

In [ ]:
from detectron2.engine import DefaultPredictor

# path to the model we just trained
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
%matplotlib inline
from src.visualization.visualize import show_img

validation_dataset = read_dateset_from(datadir, "val")

for record in random.sample(validation_dataset, 1):
    im = cv2.imread(record["file_name"])
    print(record["file_name"])
    # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    outputs = predictor(im)

    # print_json(record, tag="record")
    # print("outputs", outputs)

    # instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    v = Visualizer(im, metadata=sketches_metadata, scale=0.5)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    show_img(out.get_image())

In [ ]:
from detectron2.evaluation import COCOEvaluator

outdir = os.path.join("reports", model_name)
os.makedirs(outdir, exist_ok=True)

test_datasets = cfg.DATASETS.TEST
evaluator = [
    COCOEvaluator(
        test_set,
        cfg,
        distributed=False,
        output_dir=outdir,
    )
    for test_set in test_datasets
]

metrics = DefaultTrainer.test(cfg, predictor.model, evaluator)

In [ ]:
from datetime import datetime

# Convert inference to a dict and remove NaN values
inference_dict = dict((k, v) for k, v in metrics.items())
metrics_file = os.path.join(cfg.OUTPUT_DIR, "metrics.json")

report = {
    "model_name": model_name,
    "training_time": training_duration,
    "training_data": datadir,
    "model_path": cfg.OUTPUT_DIR,
    "config_file": cfg_file,
    "inference": inference_dict,
    "metrics_file": metrics_file,
    "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
}

report_file = f"reports/{model_name}.json"
write_json(report_file, report)

print_json(report)

In [ ]:
print(f"Model name: '{model_name}', training time: {training_duration / 60:.0f} minutes")